# US-Ignite Portland Notebook

## Portland Datasets

Portland:

Building FootPrints: http://gis-pdx.opendata.arcgis.com/datasets/building-footprints
GeoJson: https://opendata.arcgis.com/datasets/1078959662ce49cca059fe2f8930c194_48.geojson

Traffic Counts: http://gis-pdx.opendata.arcgis.com/datasets/traffic-volume-counts
GeoJson: https://opendata.arcgis.com/datasets/6ba5258ffea34e878168ddc8cf34f7e3_250.geojson

Buildings Elevations: http://gis-pdx.opendata.arcgis.com/datasets/935e774546cf4383a881a9c12c182e6e_184
GeoJson: https://opendata.arcgis.com/datasets/935e774546cf4383a881a9c12c182e6e_184.geojson

Residential Building Permits: http://gis-pdx.opendata.arcgis.com/datasets/797a38ddb60a471392ea9134448ad6c2_89
GeoJson: https://opendata.arcgis.com/datasets/797a38ddb60a471392ea9134448ad6c2_89.geojson

Busineess Districts: http://gis-pdx.opendata.arcgis.com/datasets/65c53b0f616a4a13b51e40c237c70050_11
GeoJson: https://opendata.arcgis.com/datasets/935e774546cf4383a881a9c12c182e6e_184.geojson

Recommended Bike Routes: http://gis-pdx.opendata.arcgis.com/datasets/recommended-bicycle-routes
GeoJson: https://opendata.arcgis.com/datasets/fcd1f0403cbf484bb46233b44d96f220_183.geojson

Transit stop peak arrival: http://gis-pdx.opendata.arcgis.com/datasets/transit-stop-peak-arrival?selectedAttribute=PMPeakArrivals
GeoJson: https://opendata.arcgis.com/datasets/9dbb8fa9235d4ac796065787599ab4ca_251.geojson

In [25]:
## Import regular libraries
from __future__ import print_function, division
import os
import json
import numpy as np
import datetime as dt
import pandas as pd
import geopandas as gpd
from geopandas.tools import overlay
import ast
import pylab as pl
import scipy as sp
import statsmodels as sm
import statsmodels.formula.api as smf
import seaborn as sns;
import shapely
from shapely import geometry
from fiona.crs import from_epsg
from sklearn import preprocessing, cluster, ensemble, linear_model
## Import Carto Libraries
import cartoframes
from cartoframes.auth import set_default_context
from cartoframes.viz import Map, Layer, Legend, Source
from cartoframes.data import Dataset

## Authenticating with Carto Server

In [34]:
cc = cartoframes.CartoContext(base_url='https://usignite-admin.carto.com/',
                              api_key='nJor0dLGngHDZnlUwqK1Ow')

## Import Datasets

In [40]:
p_tc = cc.read('traffic_volume_counts_1')


In [41]:
p_tc.head(6)

,the_geom,objectid,countid,channelid,locationdesc,bound,startdate,startday,starttime,enddate,...,channelnum,conditions,comment,duration,intervallen,deviceref,locationid,locationclass,countlocdesc,counttype
cartodb_id,,,,,,,,,,,,,,,,,,,,,
92,0101000020E6100000CCBA06A36FAC5EC05937837F6AC9...,92.0,190321100B.VL2,521037.0,N BRYANT ST E of BURRAGE AVE,E,2019-03-21 00:00:00+00:00,THU,2019-08-04 11:00:00,2019-03-22 00:00:00+00:00,...,1.0,NaN,ONLY BIKES @ 2611,97.0,15.0,NaN,LEG6832,NODELEG,N BRYANT ST E of N BURRAGE AVE,VOLUME
1025,0101000020E6100000FA9F33EE5FA65EC01415F05D1CC3...,1025.0,16030174.VL2,516975.0,NE 62ND AVE N of DAVIS ST,S,2016-03-01 00:00:00+00:00,TUE,2019-08-04 15:00:00,2016-03-03 00:00:00+00:00,...,2.0,NaN,WITH BIKES,184.0,15.0,NaN,LEG22325,NODELEG,NE 62ND AVE N of NE DAVIS ST,VOLUME
2064,0101000020E610000063C3A7C182A65EC0A81773441CC5...,2064.0,17020671.VL2,518250.0,NE SACRAMENTO ST W of 60TH AVE,W,2017-02-06 00:00:00+00:00,MON,2019-08-04 12:00:00,2017-02-08 00:00:00+00:00,...,2.0,NaN,WITH BIKES A,208.0,15.0,NaN,LEG32674,NODELEG,NE SACRAMENTO ST W of NE 60TH AVE,VOLUME
5830,0101000020E61000001DA7564EF0AD5EC04BB56C50B4BA...,5830.0,14092911.VL2,515082.0,SW ALICE ST E of 40TH AVE,W,2014-09-29 00:00:00+00:00,MON,2019-08-04 12:45:00,2014-10-01 00:00:00+00:00,...,2.0,NaN,@ 3822,200.0,15.0,NaN,LEG67146,NODELEG,SW ALICE ST E of SW 40TH AVE,VOLUME
6191,0101000020E61000006FB7BB8DE1AD5EC0C95DDF5276CB...,6191.0,190128103.VL2,520846.0,N FISKE AVE S of N NEWARK ST,N,2019-01-28 00:00:00+00:00,MON,2019-08-04 13:30:00,2019-02-01 00:00:00+00:00,...,1.0,NaN,@ 9219,390.0,15.0,NaN,LEG90131,NODELEG,N FISKE AVE S of N NEWARK ST,VOLUME
1844,0101000020E6100000461191B808A25EC0281EBE19AAC8...,1844.0,1901106.VL1,520807.0,NE MARINE DR E of 122ND AVE,W,2019-01-14 00:00:00+00:00,MON,2019-08-04 12:00:00,2019-01-16 00:00:00+00:00,...,1.0,NaN,NaN,204.0,15.0,NaN,LEG30871,NODELEG,NE MARINE DR E of NE 122ND AVE,VOLUME


In [43]:
p_tc.columns

Index(['the_geom', 'objectid', 'countid', 'channelid', 'locationdesc', 'bound',
       'startdate', 'startday', 'starttime', 'enddate', 'endday', 'endtime',
       'adtvolume', 'amvolume', 'ampkhrvol', 'ampkhrtime', 'ampkhrfactor',
       'pmvolume', 'pmpkhrvol', 'pmpkhrtime', 'pmpkhrfactor', 'excepttype',
       'numchannels', 'channelnum', 'conditions', 'comment', 'duration',
       'intervallen', 'deviceref', 'locationid', 'locationclass',
       'countlocdesc', 'counttype'],
      dtype='object')

In [44]:
p_tc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6670 entries, 92 to 6670
Data columns (total 33 columns):
the_geom         6670 non-null object
objectid         6670 non-null float64
countid          6670 non-null object
channelid        6670 non-null float64
locationdesc     6670 non-null object
bound            6670 non-null object
startdate        6670 non-null datetime64[ns, UTC]
startday         6670 non-null object
starttime        6670 non-null datetime64[ns]
enddate          6670 non-null datetime64[ns, UTC]
endday           6670 non-null object
endtime          6670 non-null datetime64[ns]
adtvolume        6670 non-null float64
amvolume         6670 non-null float64
ampkhrvol        6670 non-null float64
ampkhrtime       6670 non-null datetime64[ns, UTC]
ampkhrfactor     6670 non-null float64
pmvolume         6670 non-null float64
pmpkhrvol        6670 non-null float64
pmpkhrtime       6670 non-null datetime64[ns, UTC]
pmpkhrfactor     6670 non-null float64
excepttype       

/home/pui-user/anaconda3/lib/python3.6/site-packages/cartoframes/data/dataset.py:70: UserWarning: Table will be named `https://opendata.arcgis.com/datasets/1078959662ce49cca059fe2f8930c194_48.geojson`
  warn('Table will be named `{}`'.format(table_name))


In [21]:
p_fp.head(6)

AttributeError: 'Dataset' object has no attribute 'head'